In [17]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [46]:
r = requests.get('https://www.malaysia-today.net/category/news/malaysia/')
soup = BeautifulSoup(r.content, "lxml")

In [47]:
a = soup.find_all('a')
a = [a_.get('href') for a_ in a if len(a_.get('href', ''))]
a = [a_ for a_ in a if 'https://www.malaysia-today.net/category/' in a_ and '/page' not in a_]
categories = list(set(a))

In [61]:
def crawl(url, filename):
    if os.path.exists(filename):
        return True
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a', {'class': 'post-url'})
    a = list(set([a_.get('href') for a_ in a if len(a_.get('href', ''))]))
    with open(filename, 'w') as fopen:
        json.dump(a, fopen)

In [53]:
!mkdir url

In [62]:
categories[0]

'https://www.malaysia-today.net/category/videos/rpk-videos/'

In [65]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for c in categories:
    t = c.split('/category/')[1].replace('/', '-')
    r = requests.get(c)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a', {'class': 'page-numbers'})
    numbers = []
    for a_ in a:
        try:
            numbers.append(int(a_.text.strip()))
        except:
            pass
    
    if len(numbers):
        max_num = max(numbers)
    else:
        max_num = 1
    
    for i in tqdm(range(1, max_num + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [(f'{c}page/{a}', 
                 os.path.join('url', f'{t}-{a}.json')) for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}  
        
        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████████| 76/76 [04:04<00:00,  3.22s/it]
